In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor

# Capstone Project - Week 1

## Function 7 - Sometimes Lazy is Best

You are now optimising six hyper-parameters of a machine learning model. Note that it is a popular and frequently used model, so maybe you could search to see if anyone else has optisized it before?

### 1.Loading the available data:

In [3]:
# Loading the baseline data
X = np.load('initial_data/function_7/initial_inputs.npy')
Y = np.load('initial_data/function_7/initial_outputs.npy')
X2 = np.load('initial_data2/function_7/initial_inputs.npy')
Y2 = np.load('initial_data2/function_7/initial_outputs.npy')

X = np.concatenate((X,X2), dtype = float)
Y = np.concatenate((Y,Y2), dtype = float)

# Precision notation
np.set_printoptions(suppress=False)

# Adding knwon inputs/outputs from previous weeks
Xn = np.array([
    [0.111111, 0.111111, 0.111111, 0.111111, 0.111111, 0.11111],
    [0.142857, 0.071428, 0.5, 0.285714, 0.214285, 0.928571],
    [0.142857, 0.142857, 0.428571, 0.285714, 0.285714, 0.928571],
    [0.142857, 0.142857, 0.463571, 0.295714, 0.275714, 0.927571],
    [0.142857, 0.042857, 0.483571, 0.245714, 0.295714, 0.917571],
    [0.142877, 0.049867, 0.473571, 0.325714, 0.295714, 0.907571],
    [0.0625,   0.125125, 0.54321,  0.312534, 0.512234, 0.812503],
    [0.125,    0.062504, 0.501321, 0.312532, 0.375144, 0.875943],
    [0.0625,   0.125125, 0.54321,  0.312534, 0.512234, 0.812503],
    [0.125,    0.062504, 0.501321, 0.312532, 0.375144, 0.875943],
    [0.125,  0.0625, 0.5, 0.3125, 0.375,  0.875 ],
    [0.25,   0.125,  0.5, 0.3125, 0.375,  0.875 ],
    [0.125,  0.0625, 0.4375, 0.3125, 0.375,  0.8125], 
    [0.1875,   0.125002, 0.437501, 0.25,     0.375001, 0.8125  ],
    [0.1875, 0.125,  0.4375, 0.25,   0.375,  0.75  ],
    [0.1875,   0.125045, 0.437501, 0.240001, 0.374322, 0.74    ],
    [0.224285, 0.295714, 0.282714, 0.001428, 0.423571, 0.999999],
    [0.187501, 0.126001, 0.423501, 0.244001, 0.394322, 0.729864],
    [0.214285, 0.142857, 0.428571, 0.285714, 0.285714, 0.714285],
    [0.215295, 0.142855, 0.428571, 0.214285, 0.357142, 0.714285]
    ])
X = np.concatenate((X,Xn), dtype = float)

Yn = np.array(
[
    0.08866875,
    1.576790525,
    1.816555758,
    1.82184433,
    1.824727812,
    1.888488858,
    1.444479382,
    2.009221979,
    1.444479382,
    2.009221979,
    2.016527831,
    2.106734375,
    2.412757739,
    2.543077457,
    2.8771398319094748,
    2.905683599254448,
    0.4369229547,
    2.811204519,
    3.190103946,
    3.004064584
])
Y = np.concatenate((Y,Yn), dtype = float)

In [4]:
print("X:\n")
print(X)
print("\n")
print("Y:\n")
print(Y)

X:

[[0.27262382 0.32449536 0.89710881 0.83295115 0.15406269 0.79586362]
 [0.54300258 0.9246939  0.34156746 0.64648585 0.71844033 0.34313266]
 [0.09083225 0.66152938 0.06593091 0.25857701 0.96345285 0.6402654 ]
 [0.11886697 0.61505494 0.90581639 0.8553003  0.41363143 0.58523563]
 [0.63021764 0.8380969  0.68001305 0.73189509 0.52673671 0.34842921]
 [0.76491917 0.25588292 0.60908422 0.21807904 0.32294277 0.09579366]
 [0.05789554 0.49167222 0.24742222 0.21811844 0.42042833 0.73096984]
 [0.19525188 0.07922665 0.55458046 0.17056682 0.01494418 0.10703171]
 [0.64230298 0.83687455 0.02179269 0.10148801 0.68307083 0.6924164 ]
 [0.78994255 0.19554501 0.57562333 0.07365919 0.25904917 0.05109986]
 [0.52849733 0.45742436 0.36009569 0.36204551 0.81689098 0.63747637]
 [0.72261522 0.01181284 0.06364591 0.16517311 0.07924415 0.35995166]
 [0.07566492 0.33450212 0.13273274 0.60831236 0.91838592 0.82233079]
 [0.94245084 0.37743962 0.48612233 0.22879108 0.08263175 0.71195755]
 [0.14864702 0.03394336 0.7288

In [5]:
y_max = np.max(Y)
print("Max output until now: ", y_max, "which corresponds to input: ", X[np.where(Y == y_max)][0])

Max output until now:  3.190103946 which corresponds to input:  [0.214285 0.142857 0.428571 0.285714 0.285714 0.714285]


### 2. Build a Bayesian Model

In [6]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

kernel_matern = 1.0 * Matern(length_scale=1.0, nu=1.5)  # Adjust nu for smoothness
gpr = GaussianProcessRegressor(kernel=kernel_matern, n_restarts_optimizer=10)
gpr.fit(X, Y)

C:\Users\guill\anaconda3\Lib\site-packages\sklearn\gaussian_process\_gpr.py:663: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=1, nu=1.5),
                         n_restarts_optimizer=10)

### 3. Acquisition Function 1 - Upper Confidence Bound

Upper-Confidence Bound action selection uses uncertainty in the action-value estimates for balancing exploration and exploitation. Since there is inherent uncertainty in the accuracy of the action-value estimates when we use a sampled set of rewards thus UCB uses uncertainty in the estimates to drive exploration.

In [7]:
import itertools as it

x1 = np.linspace(0, 1, 15)

list(it.product(['1','2','3','4',], ['a', 'b','c','d']))

dim = 6
X_grid = np.fromiter(it.chain(*it.product(x1, repeat=dim)), dtype=float).reshape(-1,dim)
mean, std = gpr.predict(X_grid, return_std = True)

ucb1 = mean + 1.16 * std

idx_max = np.argmax(ucb1)
next_query = X_grid[idx_max]
print("UCB - Next Query - Idea 1: ", next_query)

UCB - Next Query - Idea 1:  [0.21428571 0.14285714 0.42857143 0.28571429 0.28571429 0.64285714]


### 4. Acquisition Function 2 - Probability of Improvement

In [8]:
from scipy.stats import norm

# Current best observed value (maximum)
f_best = np.max(Y)

# Compute PI values for candidate points
def compute_pi(x):
    mu, sigma = gpr.predict(X_grid, return_std = True)
    z = (mu - f_best) / sigma
    pi = norm.cdf(z)
    return pi

pi_values = compute_pi(X_grid)

# Choose the next point with maximum PI value
next_idx = np.argmax(pi_values)
next_query = X_grid[next_idx]

print("PI - Next Query - Idea 1: ", next_query)

PI - Next Query - Idea 1:  [0.21428571 0.14285714 0.42857143 0.28571429 0.28571429 0.71428571]


### 5. Acquisition Function - Thompson Sampling

In [10]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.stats import norm

# Define the GP kernel (you can choose other kernels as well)
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

# Fit the GP model to your data
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, Y)

# Compute Thompson Sampling values for candidate points
def compute_thompson(x):
    mu, sigma = gpr.predict([x], return_std=True)
    sample = np.random.normal(mu, sigma)
    return sample

thompson_values = [compute_thompson(x) for x in X_grid]

# Choose the next point with maximum Thompson Sampling value
next_idx = np.argmax(thompson_values)
next_query = X_grid[next_idx]

print("Thompson Sampling - Next Query: ", next_query)


Thompson Sampling - Next Query:  [0.21428571 0.28571429 0.28571429 0.07142857 0.42857143 1.        ]


### 6. Acquisition Function - Bayesian Expected Losses

In [11]:
# Define the GP kernel (you can choose other kernels as well)
kernel = C(1.0, (1e-3, 1e3)) * RBF(1.0, (1e-2, 1e2))

# Fit the GP model to your data
gpr = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
gpr.fit(X, Y)

# Compute the expected loss for candidate points
def compute_expected_loss(x):
    mu, sigma = gpr.predict([x], return_std=True)
    z = (mu - f_best) / sigma
    expected_loss = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return expected_loss

# We re-use X_grid due to computational needs
expected_loss_values = [compute_expected_loss(x) for x in X_grid]

# Choose the next point with minimum expected loss
next_idx = np.argmin(expected_loss_values)
next_query = X_grid[next_idx]

print("Bayesian Expected Loss - Next Query: ", next_query)


KeyboardInterrupt



### 7. Acquisition Function - Expected Improvement

In [7]:
# Compute Expected Improvement for candidate points
def compute_expected_improvement(x):
    mu, sigma = gpr.predict([x], return_std=True)
    f_best = np.max(Y)
    z = (mu - f_best) / sigma
    ei = (mu - f_best) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei

ei_values = [compute_expected_improvement(x) for x in X_grid]

# Choose the next point with maximum EI value
next_idx = np.argmax(ei_values)
next_query = X_grid[next_idx]

print("Expected Improvement - Next Query: ", next_query)


KeyboardInterrupt: 

Doing this every week, we expect to be able to find the first of the two modes! This should would be enough to provide a good solution to the problem. To really verify if we found the maximum, we would need to find a second mode, however, we may be unlucky and simply never find it. This is an important part of Machine Learning: in theory we might want optimal, perfect solutions, but in practice most of the advancements can be done with simple solutions and careful consideration of the data available to us.